# Text to spikes

## Imports

In [48]:
import pandas as pd, numpy as np

# Vectorizadores
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Import librería para simular SNN
from brian2 import *

## Data

In [49]:
data_path = '../../Author profiling/Mental-dissorders/Data/hwxpi/clean/'
train = pd.read_csv(f'{data_path}hwxpis_train.csv')
dev = pd.read_csv(f'{data_path}hwxpis_dev.csv')
test = pd.read_csv(f'{data_path}hwxpis_test.csv')

In [50]:
train.head()

,ID,raw_text,raw_text_no_lbls,corrected_text,corrected_text_no_lbls,Extraversion,Agreeableness,Conscientiousness,Neuroticism,Openness
0,DSC_0085,Fuí MISSPELLED con mí MISSPELLED familia a Ve...,"Fuí con mí familia a Veracruz , me la pase m...",Fuí MISSPELLED con mí MISSPELLED familia a Ve...,"Fui con mi familia a Veracruz , me la pase m...",0,1,1,0,0
1,DSC_0084,- Mi entrada a la Universidad - Creación de...,- Mi entrada a la Universidad - Creación de...,- Mi entrada a la Universidad - Creación de...,- Mi entrada a la Universidad - Creación de...,1,1,1,0,0
2,DSC_0087,Nací en Guanajuato pero me mudé a México cuand...,Nací en Guanajuato pero me mudé a México cuand...,Nací en Guanajuato pero me mudé a México cuand...,Nací en Guanajuato pero me mudé a México cuand...,1,0,0,1,0
3,DSC_0086,Entrar a la universidad y crear mi propia empr...,Entrar a la universidad y crear mi propia empr...,Entrar a la universidad y crear mi propia empr...,Entrar a la universidad y crear mi propia empr...,0,1,1,0,1
4,DSC_0081,Paso por un momento confuso mi familia me apo...,Paso por un momento confuso mi familia me apo...,Paso por un momento confuso mi familia me apo...,Paso por un momento confuso mi familia me apo...,0,0,0,0,1


## Process

### 1.- Transform text into a Tf-Idf matrix

In [134]:
# create a tf-idf word vectorizer
tfidf = TfidfVectorizer()
tfidf_raw_text = tfidf.fit_transform(train.raw_text).toarray()
vocabulary = tfidf

In [135]:
len(tfidf_raw_text), len(tfidf_raw_text[0])

(418, 5150)

In [136]:
# Obten los valores a partir de la columna 4 en adelante de train y guarda cada linea como una lista, de tal manera que  y_train sea una lista de listas
y_train = train.iloc[:, 5:].values.tolist()

In [69]:
y_train

[[0, 1, 1, 0, 0],
 [1, 1, 1, 0, 0],
 [1, 0, 0, 1, 0],
 [0, 1, 1, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0],
 [1, 1, 1, 0, 1],
 [0, 0, 0, 1, 1],
 [1, 1, 1, 1, 1],
 [0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0],
 [0, 1, 1, 1, 1],
 [0, 1, 0, 0, 1],
 [1, 1, 1, 1, 1],
 [1, 0, 0, 1, 1],
 [0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0],
 [1, 0, 0, 1, 0],
 [1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0],
 [1, 0, 0, 1, 1],
 [1, 0, 1, 0, 1],
 [0, 0, 0, 0, 0],
 [1, 1, 1, 1, 0],
 [0, 0, 1, 1, 0],
 [1, 1, 1, 0, 0],
 [0, 0, 1, 1, 0],
 [0, 0, 1, 1, 0],
 [1, 1, 1, 0, 0],
 [1, 0, 1, 1, 0],
 [0, 1, 0, 0, 0],
 [0, 1, 0, 0, 1],
 [0, 1, 1, 0, 0],
 [0, 0, 0, 1, 0],
 [0, 1, 0, 0, 1],
 [0, 1, 0, 0, 0],
 [1, 0, 1, 0, 0],
 [0, 1, 1, 0, 0],
 [1, 1, 1, 0, 1],
 [0, 0, 0, 1, 0],
 [0, 0, 1, 1, 0],
 [1, 0, 1, 1, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0],
 [0, 1, 1, 1, 0],
 [1, 0, 0, 1, 1],
 [1, 0, 0, 0, 0],
 [1, 1, 1, 0, 0],
 [0, 1, 0, 1, 1],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0],
 [0, 1, 0,

In [137]:
# Obten los indices de y_train donde el primer valor de cada vector sea igual a 1
y_train_extraversion = [i for i, x in enumerate(y_train) if x[0] == 1]

In [138]:
print(y_train_extraversion)

[1, 2, 8, 10, 15, 16, 18, 19, 20, 23, 24, 26, 28, 31, 32, 39, 41, 44, 49, 50, 51, 56, 57, 58, 59, 64, 67, 68, 69, 72, 75, 78, 80, 81, 82, 83, 85, 86, 87, 88, 90, 91, 94, 101, 103, 104, 106, 107, 108, 112, 114, 119, 128, 132, 134, 135, 138, 139, 142, 143, 145, 148, 154, 158, 161, 162, 170, 171, 174, 175, 176, 178, 179, 182, 184, 186, 191, 195, 196, 197, 199, 201, 205, 206, 207, 211, 213, 214, 215, 219, 223, 224, 227, 229, 231, 232, 234, 239, 241, 244, 246, 248, 254, 258, 260, 261, 268, 270, 272, 273, 274, 278, 281, 282, 283, 286, 289, 290, 291, 299, 301, 302, 303, 304, 305, 306, 310, 312, 314, 315, 316, 317, 318, 320, 322, 324, 325, 328, 330, 333, 334, 336, 337, 338, 340, 346, 347, 350, 351, 352, 355, 356, 358, 359, 361, 364, 367, 369, 370, 372, 373, 374, 375, 377, 378, 381, 382, 386, 387, 389, 392, 394, 395, 396, 397, 398, 399, 400, 401, 404, 408, 409, 410, 415, 416, 417]


## Ejemplo para una predecir una sola clase

In [139]:
# Obtener los valores de tfidf_raw_text que corresponden a los indices de y_train_extraversion
tfidf_raw_text_extraversion = [tfidf_raw_text[i] for i in y_train_extraversion]
len(tfidf_raw_text_extraversion), len(tfidf_raw_text_extraversion[0]), len(y_train_extraversion)

(186, 5150, 186)

In [97]:
186*3

558

In [157]:
from sklearn.preprocessing import normalize
tfidf_raw_text_extraversion = normalize(tfidf_raw_text_extraversion, norm='l1', axis=1)
tfidf_raw_text_extraversion[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [144]:
# Escala los valores TF-IDF para que sean apropiados como tasas de disparo (en Hz)
scale_factor = 100  # Este factor puede necesitar ajuste
tfidf_rates = tfidf_raw_text_extraversion * scale_factor

In [146]:
tfidf_rates[0]

[0. 0. 0. ... 0. 0. 0.]


In [147]:
# Asegúrate de que las tasas de disparo estén dentro de un rango razonable
# Por ejemplo, las neuronas raramente disparan a más de 100 Hz
max_rate = 100.0
tfidf_rates = np.clip(tfidf_rates, 0, max_rate)

## Ejemplo chido

In [124]:
len(tfidf_raw_text_extraversion[0])
tfidf_raw_text_extraversion

5150

## Ejemplo com todas las categorías

### 2.- Transform Tf-Idf matrix into spikes trains

### 2.1 Transformación de entrada

Para transformar una representación vectorial en trenes de picos uno, el tiempo de presentación $t_{p}$ que establece durante cuánto tiempo se presenta cada documento a la red, y el intervalo de tiempo entre dos presentaciones consecutivas $\Delta t_{p}$, debe ser definido. Es necesario un período de intervalo de tiempo, sin ningún estímulo de entrada, para eliminar la interferencia entre documentos y permitir que los parámetros dinámicos del sistema decaigan y "estén listos" para la siguiente entrada.

Técnicamente, para un documento dado $d_{i}$, representado como un vector de pesos de dimensión M $w_{ij}$, para cada peso $w_{ij}$ en cada milisegundo de presentación del documento se genera un pico con probabilidad proporcional a $w_{ij}$. Gracias a este procedimiento, finalmente obtenemos una representación del texto con picos.

En nuestros experimentos, cada documento se presenta para $t_{p}=600[ms]$ y $\Delta t_{p} = 300[ms]$, y el coeficiente de proporcionalidad $\alpha$ se establece en 1,5.

Para una mejor aclaración, consideremos un ejemplo simple y supongamos que para una palabra béisbol el peso correspondiente $w_{ij}$ en algún documento $d_{i}$ es igual a 0,1. Entonces, por cada milisegundo de tiempo de presentación, la probabilidad de emitir un pico P(pico|béisbol) es igual a $\alpha * 0,1 = 0,15$. Por lo tanto, se espera que se generen, en promedio, 90 picos durante un tiempo de presentación de 600 [ms].

In [53]:
# Parámetros de la simulación
tp = 600  # Tiempo de presentación en milisegundos
delta_tp = 300  # Intervalo de tiempo entre presentaciones en milisegundos
alpha = 1.5  # Coeficiente de proporcionalidad

# Función para transformar matriz Tf-Idf a trenes de picos teniendo en cuenta el delta_tp
def tfidf_to_spike_trains(tfidf_matrix):
    documents_spike_trains = []  # Almacenará los trenes de picos de todos los documentos
    
    for doc_idx, doc_weights in enumerate(tfidf_matrix):
        doc_spike_trains = []  # Almacena los trenes de picos para el documento actual
        
        for weight in doc_weights:
            p_spike = alpha * weight  # Probabilidad de emitir un pico
            spikes = np.random.rand(tp) < p_spike
            doc_spike_trains.append(spikes)
        
        documents_spike_trains.append(doc_spike_trains)
        
        # Si no es el último documento, añade un periodo de inactividad (delta_tp)
        if doc_idx < len(tfidf_matrix) - 1:
            inactivity_period = np.zeros((len(doc_weights), delta_tp), dtype=bool)
            documents_spike_trains.append(inactivity_period)
    
    return documents_spike_trains


In [54]:
# Ejemplo de matriz Tf-Idf
tfidf_matrix = np.array([[0.1, 0.2],  # Documento 1
                         [0.05, 0.3]])  # Documento 2

# Generar trenes de picos
spike_trains = tfidf_to_spike_trains(tfidf_matrix)

In [55]:
print(np.sum(spike_trains[0][0]))

91


In [56]:
train_spike_trains = tfidf_to_spike_trains(tfidf_raw_text)
print(len(train_spike_trains))

835


In [ ]:
# Import librería para simular SNN
from brian2 import *

start_scope()

# Asumiendo que tfidf_matrix ya está definida y es accesible
tfidf_matrix = subsets_tfidf[0]
M = tfidf_matrix.shape[1]  # Número de neuronas en la primera capa, basado en la cantidad de características TF-IDF
N = 1  # Neurona inhibitoria
alpha = 1.5  # Coeficiente de proporcionalidad para las tasas de disparo

# Parámetros del modelo LIF
tau = 100*ms
u_rest = -65*mV
u_exc = 0*mV
u_inh = -90*mV
u_th = -52*mV
t_ref = 3*ms
tau_e = 2*ms
tau_i = 2*ms

# Modelo de neurona LIF
eqs = '''
                du/dt = ((u_rest - u) + g_e*(u_exc - u) + g_i*(u_inh - u)) / tau : volt (unless refractory)
                dg_e/dt = -g_e / tau_e : 1
                dg_i/dt = -g_i / tau_i : 1
                '''

# Creación de grupos de neuronas
G = NeuronGroup(M, eqs, threshold='u > u_th', reset='u = u_rest', refractory=t_ref, method='euler')
I = NeuronGroup(N, eqs, threshold='u > u_th', reset='u = u_rest', refractory=t_ref, method='euler')

G.u = u_rest
I.u = u_rest

# Sinapsis
S_ei = Synapses(G, I, on_pre='g_e += 0.2')
S_ei.connect()

S_ie = Synapses(I, G, on_pre='g_i += 0.5')
S_ie.connect()

# Corrección: Renombrar A_pre y R_post para evitar el error
# Plasticidad STDP
eqs_plasticity = '''
                dA/dt = -A / tau_A : 1 (event-driven)
                dR/dt = -R / tau_R : 1 (event-driven)
                w : 1
                '''
on_pre = '''
                A += 1
                w = clip(w + (n * (A - (R + 0.1) * w)), 0, w_max)
                '''
on_post = '''
                R += 1
                '''

tau_A = 5*ms  # Constante de tiempo de traza presináptica
tau_R = 70*ms  # Constante de tiempo de traza postsináptica
n = 0.01  # Constante de aprendizaje
w_max = 1  # Peso máximo

S_ee = Synapses(G, G, model=eqs_plasticity, on_pre=on_pre, on_post=on_post)
S_ee.connect(condition='i != j')

# Inicialización de pesos sinápticos con una distribución aleatoria
w_init = 0.1  # Peso inicial máximo, ajusta según sea necesario para tu modelo
S_ee.w = 'rand() * w_init'

# Simulación para múltiples documentos
n_documentos = len(tfidf_matrix)  # Número de documentos

# Crear explícitamente un objeto Network
net = Network()
net.add(G)
net.add(I)
net.add(S_ei)
net.add(S_ie)
net.add(S_ee)

# Ahora, dentro de tu bucle para procesar cada documento
for doc_idx in range(n_documentos):
    rates = tfidf_matrix[doc_idx] * alpha * Hz
    P = PoissonGroup(M, rates=rates)
    S_input = Synapses(P, G, on_pre='g_e += 0.1')
    S_input.connect(j='i')
    
    # Agregar el grupo Poisson y las sinapsis a la red para este documento
    net.add(P)
    net.add(S_input)
    
    # Monitores (recreados para cada documento para evitar MagicError)
    M_spikes = SpikeMonitor(G)
    M_v = StateMonitor(G, 'v', record=True)
    
    # Agregar los monitores a la red
    net.add(M_spikes)
    net.add(M_v)
    
    # Ejecutar la simulación para este documento
    net.run(600*ms)

    # Visualización
    figure(figsize=(12, 6))
    subplot(121)
    plot(M_spikes.t/ms, M_spikes.i, '.k')
    xlim(0, 600)
    xlabel('Time (ms)')
    ylabel('Neuron index')
    
    subplot(122)
    plot(M_v.t/ms, M_v.u[0]/mV)
    xlabel('Time (ms)')
    ylabel('Membrane potential (mV)')
    
    show()

    # Importante: borra los monitores y el grupo Poisson de la red antes de la próxima iteración
    net.remove(M_spikes)
    net.remove(M_v)
    net.remove(P)
    net.remove(S_input)

### 2.2 Arquitectura y dinámica del codificador de picos

Un codificador de picos es el elemento clave del método propuesto. El codificador, presentado en la figura, es un SNN de dos capas equipado con una neurona inhibidora adicional. La primera capa contiene M neuronas (indicadas por círculos azules) y cada una de ellas representa una palabra $t_{j}$ del diccionario T. La dinámica neuronal se define por los trenes de púas generados en función de los pesos $w_{ij}$ correspondiente a documentos $d_i, i=1,...,K$. Las neuronas que representan palabras que son estadísticamente más relevantes para un documento en particular emiten un mayor número de picos, según la medida TF-IDF elegida. Los trenes de puntas para cada neurona se presentan en la figura como una fila de líneas verticales cortas.

En el cerebro, los picos se transmiten entre neuronas a través de conexiones sinápticas. Una neurona que genera un pico se llama neurona presináptica , mientras que una neurona objetivo (receptora de picos) es una neurona postsináptica . En la arquitectura SNN propuesta (figura), se utilizan dos tipos diferentes de conexiones sinápticas: las excitadoras y las inhibidoras . Los picos transmitidos a través de conexiones excitadoras (indicadas por círculos verdes en la figura) provocan la activación de la neurona postsináptica, mientras que los impulsos que viajan a través de conexiones inhibidoras (círculos rojos en la figura) obstaculizan la actividad de la neurona postsináptica . Cada vez que se activa una neurona codificadora, sus pesos se modifican de acuerdo con la regla de aprendizaje propuesta. La neurona envía simultáneamente una señal de solicitud de inhibición a la neurona inhibidora y la activa. Luego, la neurona inhibidora suprime la actividad de todas las neuronas de la capa de salida del codificador mediante una conexión inhibidora recursiva (círculos rojos). La arquitectura propuesta satisface el paradigma de aprendizaje competitivo con una estrategia en la que el ganador se lo lleva todo (WTA).

En este trabajo consideramos un modelo neuronal biológicamente plausible conocido como integración con fugas y fuego (LIF). La dinámica de dicha neurona se describe en términos de cambios de su potencial de membrana (MP). Si la neurona no recibe ningún pico, su potencial está cerca del valor de $u_{rest}=-65[mV]$ conocido como potencial de membrana en reposo. Cuando la neurona recibe picos transmitidos a través de sinapsis excitadoras, el MP se mueve hacia el potencial de equilibrio excitador, $u_{exc}=0[mV]$. Cuando se transmiten muchas señales simultáneamente a través de sinapsis excitadoras, el MP aumenta y en algún momento puede alcanzar un valor umbral de $u_{th}=-52[mV]$, en cuyo caso la neurona se dispara. Después de dispararse, la neurona restablece su MP a $u_{rest}$ y se vuelve inactivo por $t_{ref}=3[ms]$ (el período refractario). En el escenario opuesto, cuando la neurona recibe picos a través de la sinapsis inhibidora, su MP se mueve hacia el potencial de equilibrio inhibidor $u_{inh}=-90[mV]$, es decir, más lejos de su valor umbral, lo que disminuye las posibilidades de disparo. La dinámica del potencial de membrana u en el modelo LIF se describe mediante la siguiente ecuación:

$$\tau \frac{du}{dt}=(u_{rest}-u)+g_{e}(u_{exec}-u)+g_{i}(u_{inh}-u)$$

dónde $g_{e}$ y $g_{i}$ denota conductancia excitadora e inhibidora, resp. y $\tau=100[ms]$ es la constante de tiempo de la membrana. los valores de $g_{e}$ y $g_{i}$ dependen de la actividad presináptica. Cada vez que se transmite una señal a través de la sinapsis, la conductancia se incrementa en el valor del peso correspondiente a esa sinapsis y luego decae con el tiempo de acuerdo con la ecuación.

$$\tau_{e} \frac{dg_{e}}{dt}=-g_{e}, \tau_{i}=\frac{dg_{i}}{dt}=-g_{i}$$

dónde $\tau_{e}=2[ms], \tau_{i}=2[ms]$, son constantes de tiempo de desintegración. En resumen, si no hay actividad presináptica, el MP converge a $u_{rest}$. De lo contrario, su valor cambia según las señales transmitidas a través de las sinapsis neuronales.

### 2.3 Plasticidad sináptica hebbiana
Utilizamos una versión modificada del proceso de aprendizaje de Spike-Timing-Dependent Plasticity (STDP). STDP es un protocolo de aprendizaje no supervisado biológicamente plausible que pertenece a la familia de métodos de aprendizaje de Hebbian (HL). En resumen, el proceso STDP resulta en un aumento del peso sináptico si el pico postsináptico se observa poco después del presináptico ('pre-antes-post'), y en una disminución del peso sináptico en el escenario opuesto ('post- antes-pre'). El esquema de aprendizaje anterior aumenta la relevancia de aquellas conexiones sinápticas que contribuyen a la activación de la neurona postsináptica y disminuye la importancia de las que no lo hacen. Modificamos STDP, es decir, omitiendo la modificación de peso en el escenario post-antes-pre e introduciendo un mecanismo de escala adicional. La plasticidad de la sinapsis excitadora $s_{ij}$ conecta una neurona presináptica $i$ de la capa de entrada con una neurona postsináptica $j$ de la capa codificadora se puede expresar de la siguiente manera:

(1)
$$\Delta s_{ij}= n(A(t)-(R(t)+0.1)s_{ij})$$

dónde: 

(2)
$$A(t)= -\tau_{A}\frac{dA(t)}{dt}, R(t)=-\tau_{R}\frac{dR(t)}{dt}$$

y $n=0.01$ es una pequeña constante de aprendizaje. En las dos ecuaciones anteriores, $A(t)$ representa una traza presináptica y $R(t)$ es un factor de escala que depende de la historia de la actividad de las neuronas postsinápticas. Cada vez que la neurona presináptica $i$ dispara $A(t)$ se establece en 1 y decae exponencialmente en el tiempo ($\tau_{A}=5[ms]$). Si la neurona postsináptica se activa justo después de la presináptica ('pre-antes-post'), $A(t)$ está cerca de 1 y el aumento de peso es alto. El otro componente de la ecuación (3) $(R(t)+0.1)s{ij}$ es una forma de escalamiento sináptico. Cada vez que la neurona postsináptica dispara $R(t)$ se incrementa en 1 y luego decae con el tiempo ($\tau_{R}=70[ms]$). La función del pequeño factor constante 0,1 es mantener la escala incluso cuando la actividad es relativamente pequeña. El propósito general del escalamiento sináptico es disminuir el peso de las sinapsis que no participan en la activación de la neurona postsináptica. Otro beneficio del escalamiento sináptico es restringir el crecimiento incontrolado del peso, lo que se puede observar en HL

### 2.4.- Procedimiento de aprendizaje

Para un determinado corpus de datos (conjunto de documentos) $D$, el procedimiento de capacitación se realiza de la siguiente manera. En primer lugar, dividimos $D$ en $s$ subconjuntos $u_{i}, i=1$, tal como se decribe en (**2.5.- Formación de conjunto de datos**). En segundo lugar, cada subconjunto $u_{i}$ se transforma en trenes de púas y se utiliza como entrada para un codificador SNN independiente $H_{i}, i=1,...,s$ compuesto por $N$ neuronas. Tenga en cuenta que cada codificador se entrena con el uso de un solo subconjunto. Una configuración de entrenamiento de este tipo permite el procesamiento de los datos de forma paralela. Otra ventaja es que esto limita el número de conexiones excitadoras por neurona, lo que reduce la complejidad computacional (el número de ecuaciones diferenciales que deben evaluarse para cada pico) como durante el entrenamiento del codificador $H_{i}$ está expuesto sólo al subconjunto respectivo, $T_{i}$ del diccionario del conjunto de entrenamiento T y el número de sus conexiones excitadoras está limitado a $|T_{i}|<|T|$. Los trenes Spike se presentan a la red cuatro veces (en cuatro épocas de entrenamiento).

Una vez completado el proceso de aprendizaje, se aplica el procedimiento de poda de conexión . Tenga en cuenta que la AS combinada con el aprendizaje competitivo debería conducir a neuronas altamente especializadas que se activan sólo para un subconjunto de entradas. La especialización de una neurona determinada depende del conjunto de sus pesos de conexión. Si la probabilidad de disparo debe ser alta para algún subconjunto particular de entradas, los pesos que representan las palabras de esas entradas deben ser altos. Los otros pesos deberían ser relativamente bajos debido al mecanismo de escalamiento sináptico. Con base en esta suposición, después del entrenamiento, para cada neurona de la capa de salida podamos $\theta$ por ciento de sus conexiones entrantes con los pesos más bajos. $\theta$ es un hiperparámetro del método evaluado empíricamente en la sección experimental.

### 3.- Configuración del experimento
Ejemplo de la implementación que quiero replicar:
El corpus (bydate) está formado por 18846 documentos distribuidos casi por igual entre veinte categorías y explícitamente divididos en dos subconjuntos: el de entrenamiento (60%) y el de prueba (40%).

El conjunto de entrenamiento se dividió en $s=11$ subconjuntos $u_{i}$ cada uno de los cuales, excepto $u_{11}$, contenido 1500 documentos. La división se realizó de forma aleatoria. En primer lugar, se barajó todo el conjunto de entrenamiento y luego se asignó consecutivamente a los subconjuntos según el orden resultante con una redundancia (superposición) de 500 documentos entre los subconjuntos vecinos. La dinámica de las neuronas spiking (incluido el mecanismo de plasticidad) se implementó utilizando el simulador BRIAN2. Biblioteca Python de aprendizaje ScikitLearn se utilizó para procesar el texto y crear la matriz TF-IDF.

La superposición entre subconjuntos posteriores fue el resultado de experimentos preliminares que sugirieron que este enfoque mejora la precisión de la clasificación. Si bien encontramos que el concepto de superposición parcial de datos es razonablemente eficiente, de ninguna manera debe considerarse como una opción óptima. La división óptima de los datos en subconjuntos de entrenamiento sigue siendo una cuestión abierta y un tema de nuestra investigación futura.


### 3.1 Proceso de evaluación
Las salidas de todos los SNN. H_i, i=1,...,1, es decir, codificaciones basadas en picos representadas como sumas de picos por documento se unieron para formar una única matriz (una representación final de texto de baja dimensión) que se evaluó en el contexto de una tarea de clasificación. La matriz unida de tasas de picos se utilizó como entrada para el clasificador de regresión logística (LR) con la precisión como medida de rendimiento.

### 3.2.- Resultados experimentales
En el primer experimento observamos más de cerca los pesos de las neuronas después del entrenamiento y la relación entre el mecanismo de inhibición y la calidad/eficacia de la representación del texto resultante. Entrenamos once codificadores SNN con 50 neuronas cada uno según el procedimiento presentado anteriormente. Después del entrenamiento, 5 neuronas del primer codificador (H_1) se tomó una muestra aleatoria y sus pesos se utilizaron para análisis posteriores.

Los gráficos muestran que cada neurona tiene un grupo de conexiones dominantes representadas por los pesos con los valores más altos: las primeras docenas de conexiones. Significa que cada neurona será activada más fácilmente por las entradas que contengan palabras correspondientes a estos pesos. Por ejemplo, la neurona 4 potencialmente producirá más picos para documentos relacionados con la religión porque sus 10 pesos más altos corresponden a las palabras 'jesús', 'dios', 'pablo', 'fe', 'ley', 'cristianos', 'cristo', 'sábado', 'pecado', 'judío' . Hemos repetido el procedimiento anterior de muestreo e inspección de pesaje varias veces y las observaciones son cualitativamente las mismas. Por razones de ahorro de espacio no los informamos en detalle.

Por lo tanto, surge la pregunta de qué tan bien se pueden codificar los documentos con el uso de neuronas entrenadas de la manera descrita anteriormente. Intuitivamente, en la práctica la calidad de la codificación puede estar relacionada con el nivel de competencia entre las neuronas en la fase de evaluación. Si el valor de inhibición se mantiene lo suficientemente alto como para satisfacer la estrategia WTA, entonces sólo se activarán unas pocas neuronas y las demás se suprimirán inmediatamente. Este escenario dará lugar a representaciones muy escasas de los documentos de entrada, con sólo unas pocas o (en casos extremos) sólo una neurona dominando al resto. Dado que las diferencias entre documentos que pertenecen a diferentes clases pueden ser sutiles, una representación tan escasa puede no ser la configuración óptima. Para comprobar la influencia del nivel de inhibición en la representación basada en picos resultante, probamos el rendimiento de los SNN entrenados H_1 – H_11 para varios niveles de inhibición ajustando el valor de las sinapsis inhibidoras de las neuronas.

Claramente, la precisión depende en gran medida del nivel de inhibición. Los mejores resultados (aprox 78%) se logran con la inhibición establecida en 0 y disminuyen rápidamente junto con el aumento de la inhibición. Para los valores de inhibición superiores a 1,5, el gráfico de precisión entra en una meseta al nivel de aproximadamente 68%. Los resultados muestran que la representación más efectiva de los documentos se genera con ausencia de inhibición durante la fase de evaluación, es decir, cuando todas las neuronas tienen las mismas posibilidades de activarse y contribuir a la representación del documento.

La segunda serie de experimentos tuvo como objetivo explorar la relación entre la eficacia de la representación de documentos y el tamaño de los codificadores. Además, se verificó la sensibilidad de los codificadores entrenados a la poda de conexiones , con respecto a su eficiencia. Siete codificadores de varios tamaños (entre 110 y 3300 neuronas) fueron entrenados, y una vez completado el entrenamiento antes de que se llevara a cabo el procedimiento de poda de conexión .

S ilustran escenarios de poda particulares y su impacto en la precisión de la clasificación para varios tamaños de codificador 99%, 90%, 80% y 50% de los pesos más débiles fueron eliminados respectivamente en los cuatro casos analizados. En general, para codificadores SNN más pequeños (entre 110 y 1100neuronas) la precisión aumenta rápidamente junto con el aumento del tamaño del codificador. Para SSN más grandes, los cambios en la precisión son más lentos y las cuatro curvas permanecen dentro del rango [77.5 %, 80.19%].

En términos del grado de poda de la conexión, los mayores cambios en la precisión (entre 63% y 69%) se observan cuando 99% de conexiones se han eliminado (la curva roja). En particular, los resultados de los codificadores con menos de 1100 neuronas demuestran que este rango de poda afecta en gran medida la precisión de la clasificación. En redes más grandes, neuronas adicionales compensan las características eliminadas por el mecanismo de poda de conexión y los resultados se acercan cada vez más a otras configuraciones de poda.

Curiosamente, para las redes de menos de 770 neuronas, las diferencias en precisión entre 50%, 80% y 90% las configuraciones de poda son insignificantes, lo que sugiere que todavía existe una redundancia de conexiones relativamente alta en las redes podadas en el rango de 50% a 80%. Aparentemente, retener tan pocos como 10% de las ponderaciones no afecta la calidad de la representación y no causa deterioro de los resultados. Este resultado se correlaciona bien con los resultados del análisis de ponderación informado anteriormente y confirma que un subconjunto significativo de conexiones es suficiente para codificar adecuadamente la entrada. El mejor resultado de la clasificación general (80.19%) fue logrado por el codificador SNN con
neuronas y nivel de poda establecidos en 90%. Demuestra que SEM puede reducir efectivamente la dimensionalidad de la entrada de texto desde el punto inicial (aprox 130000) (del tamaño de 20 grupos de noticias que entrenan vocabulario) al tamaño de 550 - 2200 y mantener la precisión de la clasificación


### 2.6.- Procedimiento de evaluación
Las salidas de todos los SNN $H_{i},i=1,...,s$, es decir, codificaciones basadas en picos representadas como sumas de picos por documento se unieron para formar una única matriz (una representación final de texto de baja dimensión) que se evaluó en el contexto de una tarea de clasificación. La matriz unida de tasas de picos se utilizó como entrada para el clasificador de regresión logística (LR) con la precisión como medida de rendimiento.